# Lecture 6

### Givens Rotations

Two by two example:

$\begin{bmatrix}\lambda & \sigma\\ -\sigma & \lambda \end{bmatrix}$
$\begin{bmatrix}M_{11} & M_{12} & \cdots & M_{1m}\\ M_{21} & M_{22} & \cdots & M_{2m} \end{bmatrix}$
$=$
$\begin{bmatrix}\lambda M_{11} + \sigma M_{21} & \cdots &  \lambda M_{1m} + \sigma M_{2m} \\
-\sigma M_{11} + \lambda M_{21} & \cdots &  -\sigma M_{1m} + \lambda M_{2m} \end{bmatrix}$

Let $-\sigma M_{11} + \lambda M_{21} = 0$ and $\sigma^2 + \lambda^2 = 1$ we can get,

$\lambda = \frac{M_{11}}{\sqrt{M_{11}^2 + M_{21}^2}}$ and $\sigma = \frac{M_{21}}{\sqrt{M_{11}^2 + M_{21}^2}}$

**HW problem**

1.Determine the computational complexity for $QR$ decomposition using 

a.Gram-Schmidt  b. modifies Gram-Schmidt  

c. Householder reflection  d.Givens Rotation for an arbitrary dense $n \times n$ matrix.

2.Compare the complexity of Householder vs Givens Rotation for a sparse matrix.

**My Answer:**

Reference: 

http://www.seas.ucla.edu/~vandenbe/133A/lectures/qr.pdf

http://www.math.usm.edu/lambers/mat610/sum10/lecture9.pdf (This one is important)

If our $A \in R^{m \times n}$

Gram-Schmidt algorithm: $2mn^2$

Modified Gram-Schmidt algorithm: $2mn^2$

Householder algorithm: $\sum_{k=1}^{n}4(m-k)(n-k) \sim 2mn^2 - 2n^3/3$

Givens Rotations: $3mn^2 - n^3$ (or 50% more than Householder QR)

In terms of sparse matrix:

We showed how to construct Givens rotations in order to rotate two elements of a column vector
so that one element would be zero, and that approximately $n^2/2$ such rotations could be used to
transform $A$ into an upper triangular matrix $R$. Because each rotation only modifies two rows of
$A$, it is possible to interchange the order of rotations that affect different rows, and thus apply
sets of rotations in parallel. This is the main reason why Givens rotations can be preferable to
Householder reflections. Other reasons are that they are easy to use when the $QR$ factorization
needs to be updated as a result of adding a row to $A$ or deleting a column of $A$. They are also
more efficient when $A$ is sparse.

If our $A \in R^{n \times m}$

- GS:  $O(nm^2)$
- MGS: $O(nm^2)$
- HR:  $O(nm^2)$
- GR:  $O(nm^2)$

If $A$ is sparse, we can save a lot of time by using Givens Rotation. Just focus on the non-zero entries.

**HW problem**

1.Implement the $QR$ decomposition using the Householder Reflections. Input Matrix $A$(full column rank). Output $Q$, $R$.(You don't need to give all the columns of $Q$)

2.Do the same job for Givens Rotations.

In [1]:
# Householder
using LinearAlgebra

function householder(a)
    """
    Computes the householder reflection
    given a nonzero vector a.
    """
    nrm_a = norm(a,2)
    nrm_a == 0 && error("Input vector is zero.")

    d = length(a)
    v = copy(a)
    v[1] = v[1] - nrm_a
    H = Matrix{Float64}(I,d,d) - 2*v*v'/dot(v,v)
    return H
end

a = rand(10)
a = [0.8147, 0.9058, 0.1270, 0.9134, 0.6324]
H = householder(a)

function householderQR(A)
    """
    Implement the householder QR decompostion.
    Input a m * n matrix with full rank.
    Output Q, R. (Don't) need to give all the columns of Q
    """
    m, n = size(A)
    r = rank(A)
    if r != min(m, n)
        println("The input matrix is not full rank")
    else
        H1 = householder(A[:,1])
        A2 = H1 * A
        Q = H1
        Ai = A2
        for i in 2:r
            Hi_hat = householder(Ai[i:end, i])
            b1 = hcat(Matrix{Float64}(I,i-1,i-1), zeros(i-1,m-i+1))
            b2 = hcat(zeros(m-i+1, i-1), Hi_hat)
            Hi = vcat(b1, b2)
            Ai = Hi * Ai
            Q = Q*Hi
        end
        R = Q'*A
    end
    return Q, R
end


m, n = 10, 4
A = rand(m, n)
Q, R = householderQR(A)
norm(A-Q*R, 2)

1.7066752288308395e-15

In [3]:
# Another example
B = [0.8147 0.9058 0.1270 0.9134 0.6324; 
     0.0975 0.2785 0.5469 0.9575 0.9649; 
     0.1576 0.9706 0.9572 0.4854 0.8003]

Q, R = householderQR(B')

([0.492667 -0.480668 … -0.641344 0.288528; 0.547757 -0.358349 … 0.468965 -0.133537; … ; 0.552353 0.339055 … 0.0385087 -0.589282; 0.382426 0.547312 … 0.212844 0.71269], [1.65365 1.14047 1.25698; -1.65615e-16 0.966095 0.634108; … ; -1.07429e-16 1.82679e-17 2.30535e-16; -1.0406e-16 -7.57776e-18 2.45559e-17])

In [4]:
Q

5×5 Array{Float64,2}:
 0.492667   -0.480668  -0.177953   -0.641344    0.288528
 0.547757   -0.358349   0.577744    0.468965   -0.133537
 0.0767997   0.475432   0.634321   -0.567419   -0.20914 
 0.552353    0.339055  -0.480846    0.0385087  -0.589282
 0.382426    0.547312  -0.0311446   0.212844    0.71269 

In [5]:
R

5×3 Array{Float64,2}:
  1.65365       1.14047      1.25698    
 -1.65615e-16   0.966095     0.634108   
 -1.71211e-16   9.86083e-18  0.881557   
 -1.07429e-16   1.82679e-17  2.30535e-16
 -1.0406e-16   -7.57776e-18  2.45559e-17

In [21]:
# Givens Rotation
function givens_rot(a,i,j)
    """
    Computes the Givens Rotation for a
    vector 'a' at indices i and j, where
    the index at j is set to zero.
    """
    d = length(a)
    (i > d || j > d) && error("Index out of range.")
    l = sqrt(a[i]^2 + a[j]^2)
    λ = a[i]/l
    σ = a[j]/l
    G = ones(d)
    G[i] = λ
    G[j] = λ
    G = diagm(0 => G)
    G[i,j] = σ
    G[j,i] = -σ
    return G
end

function givensQR(A)
    """
    Implement Givens Rotation QR decompostion.
    Input a m * n full rank matrix. 
    Output Q and R.
    """
    m, n = size(A)
    r = rank(A)
    if r != min(m, n)
        println("The input matrix is not full rank")
    else
        Q = Matrix{Float64}(I,m,m)
        R = A
        for j in 1:n
            for i in m:-1:j+1
                Gij = givens_rot(A[:,j],i-1,i)
                A = Gij * A
                R = Gij * R
                Q = Q * Gij'
            end
        end
    end
    return Q, R
end

givensQR (generic function with 1 method)

In [22]:
m, n = 10, 4
A = rand(m, n)
Q, R = givensQR(A)
norm(A-Q*R, 2)

1.2136771230827846e-15

In [23]:
Q

10×10 Array{Float64,2}:
 0.302352    0.0474027    0.0809427  …   0.0         0.0        0.0     
 0.114335   -0.00772774   0.159959       0.0         0.0        0.0     
 0.0505762   0.760678     0.373474       0.0         0.0        0.0     
 0.0881243   0.0540396    0.392142       0.819824    0.0        0.0     
 0.0508729  -0.0433308    0.248238      -0.1961      0.539549   0.0     
 0.614501    0.203963    -0.58494    …   0.207804    0.223641  -0.340651
 0.450999   -0.303406     0.100999       0.0476578   0.230796   0.699681
 0.380294   -0.358637     0.275878      -0.0471911  -0.608093  -0.295845
 0.194364   -0.154889     0.426649      -0.288764    0.368262  -0.474744
 0.345234    0.357768     0.0498767     -0.397956   -0.316561   0.285476

In [24]:
R

10×4 Array{Float64,2}:
  1.53643       1.17368       1.40032       1.10585    
 -4.29249e-18   1.20479       0.669675      0.523554   
  5.39971e-17  -1.95748e-17   0.984912      0.935952   
  1.67049e-17   2.61778e-17   2.42841e-17   0.881137   
  7.1136e-18    9.37192e-18   8.41057e-18  -3.01749e-17
 -2.56852e-17  -1.22524e-19  -1.27006e-17   9.0221e-18 
 -1.61313e-17   5.32717e-18  -2.01995e-20  -4.78946e-18
 -3.22957e-17   1.64909e-18  -2.08466e-17   1.03788e-19
 -7.73623e-17  -2.63898e-17  -1.49121e-18   1.04439e-17
 -1.13284e-17   1.00983e-17   3.13004e-17   1.04586e-17

### "Large" data Least Squares

#### Givens rotation and large data least squares

Given $A \in \mathbb{R}^{n \times m}$ where $n$ is too large to fit $A$ into memory, i.e. many data points, Gentlemen's incremental QR decomposition can be used. With the solution to the linear model $(A'A)^{-1}A'b = R^{-1}(Q'b)$, QR decomposition of $\begin{bmatrix}A & b \\ \end{bmatrix}$ gives:

\begin{align}
\begin{bmatrix}R & Q'b \\ 0 & S \\ \end{bmatrix}
\end{align}

1. Doing the QR decomposition on the first $m+1$ rows of $A$ gives:

\begin{align}
\begin{bmatrix} \tilde{R}_{m+1} & \tilde{Q}_{m+1}'b_{m+1} \\ 0 & S_{m+1} \\ \end{bmatrix}
\end{align}

2. Add the next row: 

\begin{align}
\begin{bmatrix} \tilde{R}_{m+1} & \tilde{Q}_{m+1}'b_{m+1} \\ 0 & S_{m+1} \\ a_{m+2} & b_{m+2} \\ \end{bmatrix}
\end{align}

3. Hit this with the right Givens rotation to get:

\begin{align}
\begin{bmatrix} \tilde{R}_{m+2} & \tilde{Q}_{m+2}'b_{m+2} \\ 0 & S_{m+2} \\ \end{bmatrix}
\end{align}

4. Keep going until all rows have been added.

### Singular Value Decomposition

**Therorem:** Suppose $A \in R^{n \times m}$ and $n \geq m$. Then $\exists U \in R^{n\times n}$ and $\exists V \in R^{m\times m}$ orthogonal and there is a diagonal matrix $\Sigma =   \begin{pmatrix}
    \sigma_{1} & & \\
    & \ddots & \\
    & & \sigma_{m} \\
    0 &\cdots &0 \\
    \vdots &\vdots &\vdots \\
    0 &\cdots &0
  \end{pmatrix}$ where $\sigma_1, \cdots, \sigma_m$ are $\sigma_1 \geq \sigma_2 \geq \cdots \sigma_m \geq 0$. S.T.
  
  $$A  = U \Sigma V^T$$

**HW problem**

(1) What happens if $m > n$

**My Answer:**
$$A = U\begin{pmatrix}
    \sigma_{1} & & & 0 & \cdots &0\\
    & \ddots & & \vdots& \vdots &\vdots\\
    & & \sigma_{n} & 0 & \cdots &0
  \end{pmatrix} V^T$$

(2) If $u$,$v$ are left and right sigular vectors corresponding to sigular value $\sigma$ show that

$$Av = \sigma u$$

$$A^Tv = \sigma v$$

**My Answer**

$$A = \sum_{i = 1}^{m} \sigma_i u_i v_i^T$$
$$Av = \sum_{i = 1}^{m} \sigma_i u_i v_i^T v = \sigma_i u_i = \sigma u$$
$$A^Tu = \sum_{i = 1}^{m} \sigma_i v_i u_i^T u = \sigma_i v_i = \sigma v$$

Since $U$ and $V$ are orthogonal matrix. 